In [45]:
import pandas as pd
dataset=pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [46]:
dataset=dataset.drop("User ID",axis=1)
dataset=pd.get_dummies(dataset,drop_first=True,dtype=int)
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [47]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [48]:
independent = dataset[['Age', 'EstimatedSalary', 'Gender_Male']]
dependent = dataset[['Purchased']]

In [49]:
dependent.value_counts()

Purchased
0            257
1            143
Name: count, dtype: int64

In [50]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test= train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [51]:
from sklearn.preprocessing import StandardScaler
Sc=StandardScaler()
X_train=Sc.fit_transform(X_train)
X_test = Sc.transform(X_test)

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid={'kernel':['poly','sigmoid','rbf','linear'],'C':[0.1,10,100,1000],'gamma':['scale','auto'],'verbose':[True]}

grid = GridSearchCV(SVC(probability=True),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid = grid.fit(X_train,Y_train)

grid_prediction = grid.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test,grid_prediction)
from sklearn.metrics import classification_report
cfr=classification_report(Y_test,grid_prediction)
print("****************************************************************************")
print(cm)
print("****************************************************************************")
print(cfr)
print("****************************************************************************")


from sklearn.metrics import roc_auc_score
rac=roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])
print(rac)

dic = grid.cv_results_
table = pd.DataFrame.from_dict(dic)
table

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[LibSVM]****************************************************************************
[[73  6]
 [ 4 37]]
****************************************************************************
              precision    recall  f1-score   support

           0       0.95      0.92      0.94        79
           1       0.86      0.90      0.88        41

    accuracy                           0.92       120
   macro avg       0.90      0.91      0.91       120
weighted avg       0.92      0.92      0.92       120

****************************************************************************
0.9654214263661625


C:\Users\nravi378\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,param_verbose,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.028097,0.002092,0.007535,0.000770,0.1,scale,poly,True,"{'C': 0.1, 'gamma': 'scale', 'kernel': 'poly',...",0.755102,0.747056,0.618196,0.890114,0.758537,0.753801,0.086069,26
1,0.027447,0.009434,0.007126,0.004987,0.1,scale,sigmoid,True,"{'C': 0.1, 'gamma': 'scale', 'kernel': 'sigmoi...",0.753521,0.721707,0.644599,0.909115,0.847662,0.775321,0.093351,23
2,0.020487,0.001720,0.015623,0.000000,0.1,scale,rbf,True,"{'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf', ...",0.855314,0.857143,0.821429,0.964572,0.963889,0.892469,0.059958,5
3,0.011089,0.006454,0.014007,0.003249,0.1,scale,linear,True,"{'C': 0.1, 'gamma': 'scale', 'kernel': 'linear...",0.812540,0.763089,0.644599,0.909115,0.888158,0.803500,0.095169,21
4,0.018634,0.003674,0.007751,0.006988,0.1,auto,poly,True,"{'C': 0.1, 'gamma': 'auto', 'kernel': 'poly', ...",0.802555,0.723577,0.618196,0.870721,0.758537,0.754717,0.084104,25
5,0.028011,0.003983,0.006251,0.007656,0.1,auto,sigmoid,True,"{'C': 0.1, 'gamma': 'auto', 'kernel': 'sigmoid...",0.753521,0.721707,0.644599,0.909115,0.847662,0.775321,0.093351,23
6,0.026502,0.006282,0.015627,0.000007,0.1,auto,rbf,True,"{'C': 0.1, 'gamma': 'auto', 'kernel': 'rbf', '...",0.855314,0.857143,0.821429,0.964572,0.963889,0.892469,0.059958,5
7,0.015627,0.000002,0.009374,0.007654,0.1,auto,linear,True,"{'C': 0.1, 'gamma': 'auto', 'kernel': 'linear'...",0.812540,0.763089,0.644599,0.909115,0.888158,0.803500,0.095169,21
8,0.026132,0.008383,0.011975,0.004481,10,scale,poly,True,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly', ...",0.833024,0.799537,0.737557,0.928571,0.888158,0.837369,0.066798,10
9,0.016180,0.004562,0.008349,0.003710,10,scale,sigmoid,True,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid...",0.769053,0.733523,0.610390,0.753871,0.759910,0.725349,0.058652,32
